In [1]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [2]:
file_path = "MJD_Test_2.hdf5"
with h5py.File(file_path, 'r') as f:
    for key in f.keys():
        print(key)
    data = np.array(f)

detector
energy_label
id
psd_label_dcr
psd_label_high_avse
psd_label_low_avse
psd_label_lq
raw_waveform
run_number
tp0


In [4]:
# Open the HDF5 file and extract 20 random sets
with h5py.File(file_path, 'r') as file:
    # Load the raw waveforms and labels
    raw_waveform = np.array(file["raw_waveform"])
    energy_label = np.array(file["energy_label"])
    psd_label_low_avse = np.array(file["psd_label_low_avse"])
    psd_label_high_avse = np.array(file["psd_label_high_avse"])
    psd_label_dcr = np.array(file["psd_label_dcr"])
    psd_label_lq = np.array(file["psd_label_lq"])
    tp0 = np.array(file["tp0"])
    detector = np.array(file["detector"])
    run_number = np.array(file["run_number"])
    id = np.array(file["id"])

    # Create arrays to hold 20 sets of data
    random_waveforms = []
    energy_values = []
    psd_low_avse_values = []
    psd_high_avse_values = []
    psd_dcr_values = []
    psd_lq_values = []
    tp0_values = []
    detector_values = []
    run_number_values = []
    id_values = []

    # Select 20 random indices
    random_indices = np.random.choice(raw_waveform.shape[0], 20, replace=False)

    # Loop through each random index and store the corresponding data
    for idx in random_indices:
        random_waveforms.append(raw_waveform[idx])
        energy_values.append(energy_label[idx])
        psd_low_avse_values.append(psd_label_low_avse[idx])
        psd_high_avse_values.append(psd_label_high_avse[idx])
        psd_dcr_values.append(psd_label_dcr[idx])
        psd_lq_values.append(psd_label_lq[idx])
        tp0_values.append(tp0[idx])
        detector_values.append(detector[idx])
        run_number_values.append(run_number[idx])
        id_values.append(id[idx])

# Convert to numpy arrays for consistency (if needed for further processing)
random_waveforms = np.array(random_waveforms)
energy_values = np.array(energy_values)
psd_low_avse_values = np.array(psd_low_avse_values)
psd_high_avse_values = np.array(psd_high_avse_values)
psd_dcr_values = np.array(psd_dcr_values)
psd_lq_values = np.array(psd_lq_values)
tp0_values = np.array(tp0_values)
detector_values = np.array(detector_values)
run_number_values = np.array(run_number_values)
id_values = np.array(id_values)

# Output random indices and corresponding values for verification (optional)
random_indices, energy_values, psd_low_avse_values, psd_high_avse_values


(array([ 9912, 22620, 10528,  6505, 40407, 24491, 11174, 54724, 46029,
        53653, 22974, 44197, 24915,  5358, 26392, 14534, 19504,  1562,
         8971, 38188]),
 array([ 239.02824733,  222.15424443,  238.45002857,  253.69755875,
         531.68282145,  242.35208279, 2612.77496932,  217.38215121,
         282.38571186,  235.11077412, 1062.58116088,  837.29340502,
         291.23617674,  567.99969476,   46.68977346,  343.78751183,
         277.41842847, 1973.3013996 ,  542.61856013,  205.10675635]),
 array([ True,  True, False,  True, False,  True,  True, False,  True,
        False, False,  True, False, False, False, False,  True, False,
        False,  True]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True]))

In [ ]:
plt.figure(figsize=(12, 8))
for i, waveform in enumerate(random_waveforms):
    start = tp0_values[i]
    peak = np.argmax(waveform)
    tdrift = (peak-start) * 0.999
    tdrift50 = (peak-start) * 0.5
    tdrift10 = (peak-start) * 0.1
    rise80 = int((peak-start) * 0.8)
    '''plt.plot(waveform[start:peak])
    plt.plot(round(rise80), waveform[rise80+start], 'yo')
    plt.plot(int(tdrift50), waveform[int(tdrift50+start)], 'ro')
    plt.plot(int(tdrift10), waveform[int(tdrift10+start)], 'go')
    plt.subplot(5, 4, i+1)  # Create a 5x4 grid for 20 plots
    plt.title(f'Waveform {i+1}')
    plt.tight_layout()
    '''
    fft_tdrift = np.fft.fft(waveform[start:peak])
    frq_tdrift = np.fft.fftfreq(len(waveform[start:peak]), d=1)
    time_tdrift = np.arange(peak-start)
    
    # Plot the magnitude of the waveform in the frequency domain
    plt.plot(frq_tdrift[:len(frq_tdrift)//10], np.abs(fft_tdrift)[:len(frq_tdrift)//10])
    plt.subplot(5, 4, i+1)  # Create a 5x4 grid for 20 plots
    plt.title(f'Waveform {i+1}')
    plt.tight_layout()
    print(lfpr(fft_tdrift, 0.075))
plt.show()

In [ ]:
for i, waveform in enumerate(random_waveforms):
    textstr = (
        f"WAVEFORM: {i+1}\n"
        f"Energy Label: {energy_values[i]}\n"
        f"PSD Label Low Avse: {psd_low_avse_values[i]}\n"
        f"PSD Label High Avse: {psd_high_avse_values[i]}\n"
        f"PSD Label DCR: {psd_dcr_values[i]}\n"
        f"PSD Label LQ: {psd_lq_values[i]}\n"
        f"Start of Rising Edge: {tp0_values[i]}\n"
        f"Detector: {detector_values[i]}\n"
        f"Run Number: {run_number_values[i]}\n"
        f"ID: {id_values[i]}"
    )
    if psd_lq_values[i] and psd_dcr_values[i] and psd_low_avse_values[i]: print(i+1)

In [ ]:
with h5py.File(file_path, 'r') as file:
    # Load the raw waveforms
    raw_waveform = np.array(file["raw_waveform"])

    # Load other labels
    energy_label = np.array(file["energy_label"])
    psd_label_low_avse = np.array(file["psd_label_low_avse"])
    psd_label_high_avse = np.array(file["psd_label_high_avse"])
    psd_label_dcr = np.array(file["psd_label_dcr"])
    psd_label_lq = np.array(file["psd_label_lq"])
    tp0 = np.array(file["tp0"])
    detector = np.array(file["detector"])
    run_number = np.array(file["run_number"])
    id = np.array(file["id"])

    # Select a random index
    random_index = np.random.choice(raw_waveform.shape[0])

    # Get the random waveform
    random_waveform = raw_waveform[random_index]

    # Access the labels for the selected index
    energy_value = energy_label[random_index]
    psd_low_avse_value = psd_label_low_avse[random_index]
    psd_high_avse_value = psd_label_high_avse[random_index]
    psd_dcr_value = psd_label_dcr[random_index]
    psd_lq_value = psd_label_lq[random_index]
    tp0_value = tp0[random_index]
    detector_value = detector[random_index]
    run_number_value = run_number[random_index]
    id_value = id[random_index]

In [ ]:
start = tp0_value
peak = np.argmax(random_waveform)
tdrift = (peak-start) * 0.999
tdrift50 = (peak-start) * 0.5
tdrift10 = (peak-start) * 0.1
tdrift, tdrift50, tdrift10, tp0_value, peak

In [ ]:
rise80 = int((peak-start) * 0.8)
plt.plot(random_waveform[start:peak])
plt.plot(rise80, random_waveform[rise80+start], 'yo')

time_window = np.arange(rise80, peak + 1)
area_under_curve = np.trapz(random_waveform[rise80:peak + 1], time_window)

average_value = np.mean(random_waveform[rise80:peak + 1])

window_width = time_window[-1] - time_window[0]
area_under_average = average_value * window_width

LQ80_region = area_under_average - area_under_curve

print(f"Area under the curve: {area_under_curve}")
print(f"Area under the average line: {area_under_average}")
print(f"LQ80 region: {LQ80_region}")

In [ ]:
plt.plot(random_waveform[start:peak])
plt.plot(round(rise80), random_waveform[rise80+start], 'yo', label='i')
plt.plot(int(tdrift50), random_waveform[int(tdrift50+start)], 'ro')
plt.plot(int(tdrift10), random_waveform[int(tdrift10+start)], 'go')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(random_waveform)
plt.title(f'Random Raw Waveform (Index: {random_index})')
plt.xlabel('Time Index (μs)')
plt.ylabel('ADC Counts')

textstr = (
    f"Energy Label: {energy_value}\n"
    f"PSD Label Low Avse: {psd_low_avse_value}\n"
    f"PSD Label High Avse: {psd_high_avse_value}\n"
    f"PSD Label DCR: {psd_dcr_value}\n"
    f"PSD Label LQ: {psd_lq_value}\n"
    f"Start of Rising Edge: {tp0_value}\n"
    f"Detector: {detector_value}\n"
    f"Run Number: {run_number_value}\n"
    f"ID: {id_value}"
)
plt.gcf().text(0.45, 0.2, textstr, fontsize=10)
plt.axvline(x=750, color='purple', linestyle='--', label='Region Start (750 μs)')
plt.axvline(x=1250, color='purple', linestyle='--', label='Region End (1250 μs)')
plt.axvline(x=tp0_value, color='orange', linestyle='--', label='Rising Edge (tp0)')
plt.plot(int(tdrift50+start), random_waveform[int(tdrift50+start)], 'ro', label='tdrift50 Point')
plt.plot(int(tdrift10+start), random_waveform[int(tdrift10+start)], 'go', label='tdrift10 Point')
plt.plot(int(tdrift+start), random_waveform[int(tdrift+start)], 'bo', label='tdrift Point')

plt.legend()
plt.show()

In [ ]:
fft_waveform = np.fft.fft(random_waveform)
frequencies = np.fft.fftfreq(len(random_waveform), d=1)  # d=1 because each index corresponds to 1 microsecond
time = np.arange(3800)
# Plot the waveform in the time domain
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, random_waveform)
plt.title("Random Waveform in Time Domain")
plt.xlabel("Time (µs)")
plt.ylabel("Amplitude")

# Plot the magnitude of the waveform in the frequency domain
plt.subplot(2, 1, 2)
plt.plot(frequencies[:len(frequencies)//2], np.abs(fft_waveform)[:len(frequencies)//2])  # Plotting only positive frequencies
plt.title("Waveform in Frequency Domain")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Magnitude")

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
fft_tdrift = np.fft.fft(random_waveform[start:peak])
frq_tdrift = np.fft.fftfreq(len(random_waveform[start:peak]), d=1)
time_tdrift = np.arange(peak-start)
# Plot the waveform in the time domain
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time_tdrift, random_waveform[start:peak])
plt.title("Random Waveform in Time Domain")
plt.xlabel("Time (µs)")
plt.ylabel("Amplitude")

# Plot the magnitude of the waveform in the frequency domain
plt.subplot(2, 1, 2)
plt.plot(frq_tdrift[:len(frq_tdrift)//2], np.abs(fft_tdrift)[:len(frq_tdrift)//2])
plt.title("Waveform in Frequency Domain")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Magnitude")

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
from scipy.optimize import curve_fit

def normalized_fourier(waveform, tp0):
    peak = np.argmax(waveform)
    frq = np.fft.fftfreq(len(waveform[tp0:peak]), d=1)
    magnitude = np.abs(np.fft.fft(waveform[tp0:peak]))
    normalized_fft = magnitude / np.max(magnitude)
    return normalized_fft, frq

def sum_of_gaussians(frq, a1, mu1, sigma1, a2, mu2, sigma2):
    return (a1 * np.exp(-0.5 * ((frq - mu1) / sigma1) ** 2) +
            a2 * np.exp(-0.5 * ((frq - mu2) / sigma2) ** 2))

fft_tdrift, frq_tdrift = normalized_fourier(random_waveform, tp0_value)
peak = np.argmax(random_waveform)
time_tdrift = np.arange(peak-tp0_value)

popt, _ = curve_fit(sum_of_gaussians, frq_tdrift, fft_tdrift, p0=[1, np.mean(frq_tdrift), np.std(frq_tdrift), 0.5, np.mean(frq_tdrift) / 2, np.std(frq_tdrift) / 2])
fitted_curve = sum_of_gaussians(frq_tdrift, *popt)

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time_tdrift, random_waveform[tp0_value:peak])
plt.title("Random Waveform in Time Domain")
plt.xlabel("Time (µs)")
plt.ylabel("Amplitude")

# Plot the magnitude of the waveform in the frequency domain
plt.subplot(2, 1, 2)
plt.plot(frq_tdrift[:len(frq_tdrift)//4], np.abs(fft_tdrift)[:len(frq_tdrift)//4])
plt.plot(np.abs(frq_tdrift[:len(frq_tdrift)//4])**2, np.abs(fft_tdrift)[:len(frq_tdrift)//4], linestyle='--', color='red')
plt.title("Waveform in Frequency Domain")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Magnitude")

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
def lfpr(frq_waveform,threshold=0.05):
    power_spectrum = np.abs(frq_waveform)**2
    low_frequency_power = np.sum(power_spectrum[frq_waveform < threshold])
    return low_frequency_power / np.sum(power_spectrum)

lfpr(fft_tdrift)